# LOADING DATASETS AND IMPORTING NECESSARY LIBRARIES AND PACKAGES

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
matches= pd.read_csv('MATCHES.CSV')
deliveries= pd.read_csv('DELIVERIES.csv')

# UNDERSTANDING OUR DATASET

In [3]:
matches.head(10)

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN
5,6,IPL-2017,Hyderabad,09-04-2017,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Sunrisers Hyderabad,0,9,Rashid Khan,"Rajiv Gandhi International Stadium, Uppal",A Deshmukh,NJ Llong,NaN
6,7,IPL-2017,Mumbai,09-04-2017,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,NaN
7,8,IPL-2017,Indore,10-04-2017,Royal Challengers Bangalore,Kings XI Punjab,Royal Challengers Bangalore,bat,normal,0,Kings XI Punjab,0,8,AR Patel,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
8,9,IPL-2017,Pune,11-04-2017,Delhi Daredevils,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Delhi Daredevils,97,0,SV Samson,Maharashtra Cricket Association Stadium,AY Dandekar,S Ravi,NaN
9,10,IPL-2017,Mumbai,12-04-2017,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,NaN


In [4]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               756 non-null    int64 
 1   Season           756 non-null    object
 2   city             749 non-null    object
 3   date             756 non-null    object
 4   team1            756 non-null    object
 5   team2            756 non-null    object
 6   toss_winner      756 non-null    object
 7   toss_decision    756 non-null    object
 8   result           756 non-null    object
 9   dl_applied       756 non-null    int64 
 10  winner           752 non-null    object
 11  win_by_runs      756 non-null    int64 
 12  win_by_wickets   756 non-null    int64 
 13  player_of_match  752 non-null    object
 14  venue            756 non-null    object
 15  umpire1          754 non-null    object
 16  umpire2          754 non-null    object
 17  umpire3          119 non-null    ob

'matches' dataframe consist of 756 matches played since the start of IPL season(2008) till 2019. It contains full information about each  match like:-


1.   This match was played in which IPL seasons(between 2008-2019).
2.   When and and in which city and stadium it was played.
3.   Between whom it was played.
4.   Who won the toss and then what they decided to do in first innnings.
5.   Whether match was played like a normal t20 game .
6.   Whether DLS method was applied or not.
7.   Who was the winnner and they won by how many runs/wickets.
8.   Who was the player of match.
9.   Who were the umpires in that match.

In [5]:
deliveries.head(10)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN
5,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,6,S Dhawan,DA Warner,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,7,S Dhawan,DA Warner,TS Mills,0,...,0,1,0,0,0,1,1,NaN,NaN,NaN
7,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,2,1,S Dhawan,DA Warner,A Choudhary,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
8,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,2,2,DA Warner,S Dhawan,A Choudhary,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
9,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,2,3,DA Warner,S Dhawan,A Choudhary,0,...,0,0,1,0,0,1,1,NaN,NaN,NaN


In [6]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          179078 non-null  int64 
 1   inning            179078 non-null  int64 
 2   batting_team      179078 non-null  object
 3   bowling_team      179078 non-null  object
 4   over              179078 non-null  int64 
 5   ball              179078 non-null  int64 
 6   batsman           179078 non-null  object
 7   non_striker       179078 non-null  object
 8   bowler            179078 non-null  object
 9   is_super_over     179078 non-null  int64 
 10  wide_runs         179078 non-null  int64 
 11  bye_runs          179078 non-null  int64 
 12  legbye_runs       179078 non-null  int64 
 13  noball_runs       179078 non-null  int64 
 14  penalty_runs      179078 non-null  int64 
 15  batsman_runs      179078 non-null  int64 
 16  extra_runs        179078 non-null  int

'deliveries' is a more roboust and bigger dataset. This dataset contain ball by ball information in each of the 756 matches. Each row is a 'ball' bowled in the match and it contains information like:-

1.   Which match is it
2.   Is it first or second innings
3.   Who is batting team and who is bowling team
4.   Which ball of which over is being bowled.
4.   Who is at striker's end and at non striker's end.
5.   Who is the bowler.
6.   Is this delivery bowled in a super over or not
7.   How many runs were made if the ball bowled was a wide delivery, or a no ball delivery, or some byes or leg byes runs were scored in that ball.
8.   What is team's total score, what is striker's score.
9.   If any player was out in that ball, then what type of dismissal he had, if he was caught out or run out then who was the fielder

# DATA PREPROCESSING AND FEATURE ENGINEERING

We want to build a IPL match predictor which tells us the winning percentage of a team while chasing a target(i.e, in second innnings) based on current match scenario. Thus the features which are important to find percentage are as follows
1.   Who is batting team and bowling team in the second innings.
2.   At what place this match is being played(as ground conditions like whats the outfield and 'home ground effect' is always there and becomes important while chasing).
3.   Whats the equation in the match
    *   What is the target
    *   How many runs are left to chase
    *   How many balls are left in the match.
    *   How many wickets are in hand.
    *   What is current run rate and required run rate.

4. Then finally whether chasing team won the match or not.

Thus we need to make a new dataset containing these new features on which our model will be trained. And these new features will be made from the two datasets we are given.

In [7]:
# 'delivery' has a column of runs of the batting team ball by ball
# thus we would use that to find scores of both the teams in their respective batting innings
total_score_df=deliveries.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
total_score_df

,match_id,inning,total_runs
0,1,1,207
1,1,2,172
2,2,1,184
3,2,2,187
4,3,1,183
...,...,...,...
1523,11413,2,170
1524,11414,1,155
1525,11414,2,162
1526,11415,1,152


In [8]:
# this would be used to find target to be chased, and then calculatng required run rate
first_innings_score_df = total_score_df[total_score_df['inning'] == 1] # take only those rows from total_score_df whose inning value is 1
first_innings_score_df

,match_id,inning,total_runs
0,1,1,207
2,2,1,184
4,3,1,183
6,4,1,163
8,5,1,157
...,...,...,...
1518,11347,1,143
1520,11412,1,136
1522,11413,1,171
1524,11414,1,155


In [9]:
final_df = matches.merge(first_innings_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
final_df

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,184
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,3,183
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,4,163
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,IPL-2019,Mumbai,05-05-2019,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi,11347,143
752,11412,IPL-2019,Chennai,07-05-2019,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould,11412,136
753,11413,IPL-2019,Visakhapatnam,08-05-2019,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN,11413,171
754,11414,IPL-2019,Visakhapatnam,10-05-2019,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin,11414,155


Now we need to change the names of team(As team name got changed) and remove those teams which are currently not playing ipl. We need to change there names and removes those teams from both the datasets and from the columns which contain their name 

In [10]:
final_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [11]:
# delhi daredevils---->delhi captials (as team name got changed)
final_df['team1'] = final_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
final_df['team2'] = final_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
final_df['toss_winner'] = final_df['toss_winner'].str.replace('Delhi Daredevils','Delhi Capitals')
final_df['winner'] = final_df['winner'].str.replace('Delhi Daredevils','Delhi Capitals')
deliveries['batting_team'] = deliveries['batting_team'].str.replace('Delhi Daredevils','Delhi Capitals')
deliveries['bowling_team'] = deliveries['bowling_team'].str.replace('Delhi Daredevils','Delhi Capitals')

# Deccan Chargers---> Sunrisers Hyderabad (as team name got changed)
final_df['team1'] = final_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
final_df['team2'] = final_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
final_df['toss_winner'] = final_df['toss_winner'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
final_df['winner'] = final_df['winner'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
deliveries['batting_team']=deliveries['batting_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
deliveries['bowling_team']=deliveries['bowling_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

# Kings XI punjab----> Punjab Kings (as team name got changed)
final_df['team1'] = final_df['team1'].str.replace('Kings XI Punjab','Punjab Kings')
final_df['team2'] = final_df['team2'].str.replace('Kings XI Punjab','Punjab Kings')
final_df['toss_winner'] = final_df['toss_winner'].str.replace('Kings XI Punjab','Punjab Kings')
final_df['winner'] = final_df['winner'].str.replace('Kings XI Punjab','Punjab Kings')
deliveries['batting_team']=deliveries['batting_team'].str.replace('Kings XI Punjab','Punjab Kings')
deliveries['bowling_team']=deliveries['bowling_team'].str.replace('Kings XI Punjab','Punjab Kings')

final_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Capitals', 'Punjab Kings',
       'Chennai Super Kings', 'Rajasthan Royals', 'Kochi Tuskers Kerala',
       'Pune Warriors', 'Rising Pune Supergiants'], dtype=object)

In [12]:
#cuurently active teams
teams= [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Delhi Capitals',
    'Punjab Kings',
    'Chennai Super Kings',
    'Rajasthan Royals',
]


# keep only those teams from all rows which are currently playing the ipl

final_df = final_df[final_df['team1'].isin(teams)]
final_df = final_df[final_df['team2'].isin(teams)]
final_df = final_df[final_df['toss_winner'].isin(teams)]
final_df = final_df[final_df['winner'].isin(teams)]

deliveries = deliveries[deliveries['batting_team'].isin(teams)]
deliveries = deliveries[deliveries['bowling_team'].isin(teams)]

In [13]:
final_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Punjab Kings', 'Delhi Capitals',
       'Mumbai Indians', 'Chennai Super Kings', 'Rajasthan Royals'],
      dtype=object)

In [14]:
deliveries['batting_team'].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Delhi Capitals', 'Kolkata Knight Riders', 'Mumbai Indians',
       'Punjab Kings', 'Chennai Super Kings', 'Rajasthan Royals'],
      dtype=object)

In [15]:
final_df.shape  
# earlier we had data of total 756 matches..now after removing those team's matches which are currently inactive, we are left with 638.

(638, 20)

Now we have to remove those matches which were effected by Duckworth-Lewis-Stern (DLS) method so that we can get data of only those matches which weren't affected by rain.

In [16]:
final_df['dl_applied'].value_counts()

0    623
1     15
Name: dl_applied, dtype: int64

In [17]:
# dl_applied=0 means match nor affected by dls
final_df = final_df[final_df['dl_applied'] == 0]
final_df

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,157
6,7,IPL-2017,Mumbai,09-04-2017,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,7,178
7,8,IPL-2017,Indore,10-04-2017,Royal Challengers Bangalore,Punjab Kings,Royal Challengers Bangalore,bat,normal,0,Punjab Kings,0,8,AR Patel,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,8,148
9,10,IPL-2017,Mumbai,12-04-2017,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,10,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,IPL-2019,Mumbai,05-05-2019,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi,11347,143
752,11412,IPL-2019,Chennai,07-05-2019,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould,11412,136
753,11413,IPL-2019,Visakhapatnam,08-05-2019,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN,11413,171
754,11414,IPL-2019,Visakhapatnam,10-05-2019,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin,11414,155


In [18]:
final_df['dl_applied'].value_counts()
# see all those matches are removed

0    623
Name: dl_applied, dtype: int64

Now we will extract 4 features from 'final_df' and merge it with 'deliveries', as deliveries contain ball by ball detail of each match, thus it woill be beneficial for us and form a new dataset called 'complete_df'

In [19]:
extracted=final_df[['match_id','city','winner','total_runs']]
extracted

,match_id,city,winner,total_runs
0,1,Hyderabad,Sunrisers Hyderabad,207
4,5,Bangalore,Royal Challengers Bangalore,157
6,7,Mumbai,Mumbai Indians,178
7,8,Indore,Punjab Kings,148
9,10,Mumbai,Mumbai Indians,158
...,...,...,...,...
751,11347,Mumbai,Mumbai Indians,143
752,11412,Chennai,Mumbai Indians,136
753,11413,Visakhapatnam,Delhi Capitals,171
754,11414,Visakhapatnam,Chennai Super Kings,155


In [20]:
complete_df=extracted.merge(deliveries,on='match_id')
complete_df
#total_run_x is first inning score
#total_run_y is ball by ball team score of 2nd innings

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
0,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,...,0,0,0,0,0,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,0,1,0,1,NaN,NaN,NaN
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,0,2,0,2,NaN,NaN,NaN
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,0,1,0,1,SR Watson,run out,KH Pandya
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,0,2,0,2,NaN,NaN,NaN


Now we have complete ball by ball data i.e, in which stadium it was bowled, who was the winner of that macth in which that ball was bowled and in what innings that ball was bowled.

But We need to take only all those balls which were bowled in 2nd innings

In [21]:
# now we want to have chasing data only(i.e, whose innings is 2)
chasing_df=complete_df[complete_df['inning']==2]
chasing_df
# now this containg ball by ball data of all the balls being bowled in the second innnings

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,0,0,2,0,2,NaN,NaN,NaN
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,0,0,4,0,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,0,1,0,1,NaN,NaN,NaN
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,0,2,0,2,NaN,NaN,NaN
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,0,1,0,1,SR Watson,run out,KH Pandya
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,0,2,0,2,NaN,NaN,NaN


Now we need to find the 2nd innings socre ball by ball ,so that we can use it to find runs left to be chased ball by ball

In [22]:
chasing_df['current_score']=chasing_df.groupby('match_id')['total_runs_y'].cumsum()
chasing_df
# in last we got current score being updated ball by ball

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\3809138297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['current_score']=chasing_df.groupby('match_id')['total_runs_y'].cumsum()


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,1,0,1,NaN,NaN,NaN,1
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,NaN,NaN,NaN,1
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,0,NaN,NaN,NaN,1
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,0,2,0,2,NaN,NaN,NaN,3
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,0,4,0,4,NaN,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,1,0,1,NaN,NaN,NaN,152
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,2,0,2,NaN,NaN,NaN,154
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,1,0,1,SR Watson,run out,KH Pandya,155
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,2,0,2,NaN,NaN,NaN,157


In [23]:
chasing_df['runs_required']=chasing_df['total_runs_x']+1-chasing_df['current_score']
chasing_df['runs_required'] = chasing_df['runs_required'].apply(lambda x: max(0, x)) # in this way if anytime we had a situation where the score was chased, but due to calculation, it comes negative, for that we put 0
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\1527724219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['runs_required']=chasing_df['total_runs_x']+1-chasing_df['current_score']
C:\Users\hp\AppData\Local\Temp\ipykernel_5172\1527724219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['runs_required'] = chasing_df['runs_required'].apply(lambda x: max(0, x)) # in this way if anytime we had a situation where the score was chased, but due to calculation, it comes negative, for that 

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,1,0,1,NaN,NaN,NaN,1,207
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,2,0,2,NaN,NaN,NaN,3,205
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,4,0,4,NaN,NaN,NaN,7,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,1,0,1,NaN,NaN,NaN,152,1
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,2,0,2,NaN,NaN,NaN,154,0
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,1,0,1,SR Watson,run out,KH Pandya,155,0
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,2,0,2,NaN,NaN,NaN,157,0


Now we have to change the definintion of over 

1) 0.1= 1th over's 1st ball(1 st ball of match)

2) 1.1= 2nd over's 1st ball(total 7 balls have been bowled)

3) 10.0= total 10 overs are completed

But here, overs start from 1 i.e, 1.1= 1st ball of match

In [24]:
chasing_df['over']=chasing_df['over']-1
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\1180073525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['over']=chasing_df['over']-1


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,0,1,0,1,NaN,NaN,NaN,1,207
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,0,2,0,2,NaN,NaN,NaN,3,205
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,0,4,0,4,NaN,NaN,NaN,7,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,0,1,0,1,NaN,NaN,NaN,152,1
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,0,2,0,2,NaN,NaN,NaN,154,0
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,0,0,1,0,1,SR Watson,run out,KH Pandya,155,0
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,0,2,0,2,NaN,NaN,NaN,157,0


In [25]:
mask = chasing_df['ball'] == 6
chasing_df.loc[mask, 'ball'] = 0
chasing_df.loc[mask, 'over'] += 1
chasing_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,0,1,0,1,NaN,NaN,NaN,1,207
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,NaN,1,207
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,0,2,0,2,NaN,NaN,NaN,3,205
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,0,4,0,4,NaN,NaN,NaN,7,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,0,1,0,1,NaN,NaN,NaN,152,1
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,0,2,0,2,NaN,NaN,NaN,154,0
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,0,0,1,0,1,SR Watson,run out,KH Pandya,155,0
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,0,2,0,2,NaN,NaN,NaN,157,0


Thus now, when going from 5th ball to 6th ball i.e, over complete, we do it like 0.5->1.0

In [26]:
# balls left to be bowled after each ball
chasing_df['balls_left']=120-(chasing_df['over']*6+chasing_df['ball'])
chasing_df['balls_left'] = chasing_df['balls_left'].apply(lambda x: max(0, x))
chasing_df
# over->1, ball->1 means 1st ball being bowled
# thus number of balls bowled=(over-1)*6+ball

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\608920208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['balls_left']=120-(chasing_df['over']*6+chasing_df['ball'])
C:\Users\hp\AppData\Local\Temp\ipykernel_5172\608920208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['balls_left'] = chasing_df['balls_left'].apply(lambda x: max(0, x))


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,1,0,1,NaN,NaN,NaN,1,207,119
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,207,118
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,207,117
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,2,0,2,NaN,NaN,NaN,3,205,116
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,4,0,4,NaN,NaN,NaN,7,201,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,1,0,1,NaN,NaN,NaN,152,1,4
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,2,0,2,NaN,NaN,NaN,154,0,3
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,0,1,0,1,SR Watson,run out,KH Pandya,155,0,2
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,2,0,2,NaN,NaN,NaN,157,0,1


Now our concern is wickets left. Here we don't directly have the data for wickets. But we do have a column player_dismissed.
Thus using that we find whether a wicket was fallen in that delivery or not

In [27]:
# filling 0 in place of those balls in which no one was dismissed(as where no one was dismissed, we have NaN)
chasing_df['player_dismissed']=chasing_df['player_dismissed'].fillna("0")
# NOW IN PLAYER DISMISSED COLUMN, WE HAVE EITHER 0, OR NAME OF PLAYER
# THUS IF WE ENCOUNTER A NAME OF PLAYER, WE REPLACE IT WITH 1(SYMBOLIZING THAT A WICKET HAS FALLEN)
chasing_df['player_dismissed']=chasing_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\287583427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['player_dismissed']=chasing_df['player_dismissed'].fillna("0")
C:\Users\hp\AppData\Local\Temp\ipykernel_5172\287583427.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['player_dismissed']=chasing_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,1,0,1,0,NaN,NaN,1,207,119
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,1,207,118
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,0,0,NaN,NaN,1,207,117
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,2,0,2,0,NaN,NaN,3,205,116
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,4,0,4,0,NaN,NaN,7,201,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,1,0,1,0,NaN,NaN,152,1,4
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,2,0,2,0,NaN,NaN,154,0,3
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,0,1,0,1,1,run out,KH Pandya,155,0,2
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,2,0,2,0,NaN,NaN,157,0,1


Still, the values 0 and 1 are string values, which can't be used for calculation of wickets. Thus we need to convert them to integer and then use them

In [28]:
chasing_df['player_dismissed']=chasing_df['player_dismissed'].astype('int') # converting to int
chasing_df['wickets_left']=10-chasing_df.groupby('match_id')['player_dismissed'].cumsum() # making a new column wickets left
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\3575004406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['player_dismissed']=chasing_df['player_dismissed'].astype('int') # converting to int
C:\Users\hp\AppData\Local\Temp\ipykernel_5172\3575004406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['wickets_left']=10-chasing_df.groupby('match_id')['player_dismissed'].cumsum() # making a new column wickets left


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left,wickets_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,1,0,1,0,NaN,NaN,1,207,119,10
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,207,118,10
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,207,117,10
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,2,0,2,0,NaN,NaN,3,205,116,10
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,4,0,4,0,NaN,NaN,7,201,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,1,0,1,0,NaN,NaN,152,1,4,5
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,2,0,2,0,NaN,NaN,154,0,3,5
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,1,0,1,1,run out,KH Pandya,155,0,2,4
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,2,0,2,0,NaN,NaN,157,0,1,4


Now we need to find current run rate

In [29]:
chasing_df['crr']=(chasing_df['current_score']*6)/(120-chasing_df['balls_left'])
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\2288517790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['crr']=(chasing_df['current_score']*6)/(120-chasing_df['balls_left'])


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left,wickets_left,crr
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,1,0,NaN,NaN,1,207,119,10,6.000000
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,0,NaN,NaN,1,207,118,10,3.000000
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,0,NaN,NaN,1,207,117,10,2.000000
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,2,0,NaN,NaN,3,205,116,10,4.500000
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,4,0,NaN,NaN,7,201,115,10,8.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,1,0,NaN,NaN,152,1,4,5,7.862069
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,2,0,NaN,NaN,154,0,3,5,7.897436
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,0,1,1,run out,KH Pandya,155,0,2,4,7.881356
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,2,0,NaN,NaN,157,0,1,4,7.915966


Now we need to find required run rate

In [30]:
chasing_df['rrr']=(chasing_df['runs_required']*6)/chasing_df['balls_left']
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\3214566136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['rrr']=(chasing_df['runs_required']*6)/chasing_df['balls_left']


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left,wickets_left,crr,rrr
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,1,0,NaN,NaN,1,207,119,10,6.000000,10.436975
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,0,NaN,NaN,1,207,118,10,3.000000,10.525424
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,0,NaN,NaN,1,207,117,10,2.000000,10.615385
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,2,0,NaN,NaN,3,205,116,10,4.500000,10.603448
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,4,0,NaN,NaN,7,201,115,10,8.400000,10.486957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,1,0,NaN,NaN,152,1,4,5,7.862069,1.500000
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,2,0,NaN,NaN,154,0,3,5,7.897436,0.000000
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,1,1,run out,KH Pandya,155,0,2,4,7.881356,0.000000
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,2,0,NaN,NaN,157,0,1,4,7.915966,0.000000


Now we need to find the result i.e, if chasing team won or not. For that, we have two columns, winners and batting team(which batted in 2nd innings). Thus if they are same then we will put 1 in that row, else 0 

In [31]:
# if chasing team won=1
# if chasing team lost=0
# thus for that we compare the 'winner' and 'batting_team' columns and see if they are same(if they are same then it means chasing team won)
chasing_df['chasing_team_won']=np.where(chasing_df['winner']==chasing_df['batting_team'],1,0)
chasing_df

C:\Users\hp\AppData\Local\Temp\ipykernel_5172\2173347523.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chasing_df['chasing_team_won']=np.where(chasing_df['winner']==chasing_df['batting_team'],1,0)


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,player_dismissed,dismissal_kind,fielder,current_score,runs_required,balls_left,wickets_left,crr,rrr,chasing_team_won
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,1,CH Gayle,...,0,NaN,NaN,1,207,119,10,6.000000,10.436975,0
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,2,Mandeep Singh,...,0,NaN,NaN,1,207,118,10,3.000000,10.525424,0
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,3,Mandeep Singh,...,0,NaN,NaN,1,207,117,10,2.000000,10.615385,0
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,4,Mandeep Singh,...,0,NaN,NaN,3,205,116,10,4.500000,10.603448,0
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,0,5,Mandeep Singh,...,0,NaN,NaN,7,201,115,10,8.400000,10.486957,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149264,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,2,RA Jadeja,...,0,NaN,NaN,152,1,4,5,7.862069,1.500000,0
149265,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,3,SR Watson,...,0,NaN,NaN,154,0,3,5,7.897436,0.000000,0
149266,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,4,SR Watson,...,1,run out,KH Pandya,155,0,2,4,7.881356,0.000000,0
149267,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,19,5,SN Thakur,...,0,NaN,NaN,157,0,1,4,7.915966,0.000000,0


Now we have all the requirements in above chasing_df. So we extract the required columns which will serve as the real dataset.

In [32]:
final_df=chasing_df[['batting_team','bowling_team','city','runs_required','balls_left','wickets_left','crr','rrr','chasing_team_won']]
final_df

,batting_team,bowling_team,city,runs_required,balls_left,wickets_left,crr,rrr,chasing_team_won
125,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,207,119,10,6.000000,10.436975,0
126,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,207,118,10,3.000000,10.525424,0
127,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,207,117,10,2.000000,10.615385,0
128,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,205,116,10,4.500000,10.603448,0
129,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,201,115,10,8.400000,10.486957,0
...,...,...,...,...,...,...,...,...,...
149264,Chennai Super Kings,Mumbai Indians,Hyderabad,1,4,5,7.862069,1.500000,0
149265,Chennai Super Kings,Mumbai Indians,Hyderabad,0,3,5,7.897436,0.000000,0
149266,Chennai Super Kings,Mumbai Indians,Hyderabad,0,2,4,7.881356,0.000000,0
149267,Chennai Super Kings,Mumbai Indians,Hyderabad,0,1,4,7.915966,0.000000,0


In [33]:
final_df.describe()

,runs_required,balls_left,wickets_left,crr,rrr,chasing_team_won
count,72386.000000,72386.000000,72386.000000,72386.000000,7.235900e+04,72386.000000
mean,93.273796,62.640055,7.535062,7.438721,inf,0.525820
std,49.995039,33.395439,2.138518,2.273106,NaN,0.499336
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,54.000000,35.000000,6.000000,6.257143,7.266361e+00,0.000000
50%,93.000000,63.000000,8.000000,7.480519,9.000000e+00,1.000000
75%,131.000000,91.000000,9.000000,8.680851,1.106250e+01,1.000000
max,250.000000,119.000000,10.000000,42.000000,inf,1.000000


So we can see that rrr hass some values which are infinite and -infinte. This happens when we have some runs to chase and no balls left. Thus we need to discard them

In [34]:
final_df=final_df[final_df['balls_left']!=0]
final_df.describe()

,runs_required,balls_left,wickets_left,crr,rrr,chasing_team_won
count,72076.000000,72076.000000,72076.000000,72076.000000,72076.000000,72076.000000
mean,93.588323,62.909471,7.555136,7.436456,10.677126,0.527540
std,49.857039,33.212996,2.118494,2.276284,12.995239,0.499244
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,54.000000,35.000000,6.000000,6.250000,7.260000,0.000000
50%,93.000000,64.000000,8.000000,7.476923,9.000000,1.000000
75%,131.000000,92.000000,9.000000,8.680851,11.023256,1.000000
max,250.000000,119.000000,10.000000,42.000000,684.000000,1.000000


Now we shuffle all the datas such that match scenario of a single match aren't together. In this way we might be able to reduce biasness

In [35]:
final_df=final_df.sample(final_df.shape[0])

In [36]:
final_df

,batting_team,bowling_team,city,runs_required,balls_left,wickets_left,crr,rrr,chasing_team_won
10811,Mumbai Indians,Kolkata Knight Riders,Kolkata,8,17,7,7.572816,2.823529,1
73692,Kolkata Knight Riders,Rajasthan Royals,Jaipur,99,73,5,5.872340,8.136986,0
110007,Rajasthan Royals,Sunrisers Hyderabad,Mumbai,134,60,7,6.800000,13.400000,0
80515,Punjab Kings,Chennai Super Kings,Chennai,17,3,5,8.717949,34.000000,0
56138,Delhi Capitals,Punjab Kings,Dharamsala,153,96,10,4.500000,9.562500,0
...,...,...,...,...,...,...,...,...,...
8872,Mumbai Indians,Chennai Super Kings,Chennai,103,46,6,8.594595,13.434783,0
129178,Sunrisers Hyderabad,Delhi Capitals,Hyderabad,80,56,9,8.156250,8.571429,1
80261,Delhi Capitals,Kolkata Knight Riders,Raipur,29,28,8,7.043478,6.214286,1
110279,Mumbai Indians,Chennai Super Kings,Chennai,59,31,7,6.741573,11.419355,1


Now we split our dataset to X and y

In [37]:
X=final_df.iloc[:,:-1]
y=final_df.iloc[:,-1]

# CREATING TRAINING AND TEST DATA

Now we split the data into training and test datasets

In [38]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [39]:
x_train

,batting_team,bowling_team,city,runs_required,balls_left,wickets_left,crr,rrr
96185,Delhi Capitals,Rajasthan Royals,Ahmedabad,82,11,1,6.605505,44.727273
120411,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata,124,83,9,7.783784,8.963855
141698,Royal Challengers Bangalore,Punjab Kings,Mohali,139,99,10,12.000000,8.424242
58318,Royal Challengers Bangalore,Chennai Super Kings,Chennai,139,65,6,7.309091,12.830769
86549,Chennai Super Kings,Mumbai Indians,Kolkata,66,30,2,5.533333,13.200000
...,...,...,...,...,...,...,...,...
1810,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata,168,113,10,4.285714,8.920354
106546,Punjab Kings,Chennai Super Kings,Chennai,138,63,5,5.789474,13.142857
48007,Kolkata Knight Riders,Chennai Super Kings,Chennai,71,57,9,7.904762,7.473684
26493,Punjab Kings,Rajasthan Royals,Durban,90,15,3,6.971429,36.000000


Now the problem is that our three columns, batting team, bowling team and city are strings. So we use OneHotEncoding on them to convert them to numerical values

In [40]:
trf = ColumnTransformer([
    ('onehot', OneHotEncoder(sparse_output=False, drop='first'), ['batting_team', 'bowling_team', 'city'])
], remainder='passthrough')

In [41]:
pipe = Pipeline([
    ('step1', trf),  # Apply ColumnTransformer
    ('step2', LogisticRegression(solver='liblinear'))  # Apply LogisticRegression
])

In [42]:
pipe.fit(x_train,y_train)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:1624: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [43]:
y_hat=pipe.predict(x_test)
accuracy=accuracy_score(y_test,y_hat)
accuracy

0.8021642619311876

In [44]:
y_hat_prob=pipe.predict_proba(x_test)
y_hat_prob
# here we get a lot smoother and very less strong predictions of match winning

array([[0.015353  , 0.984647  ],
       [0.76147764, 0.23852236],
       [0.14540302, 0.85459698],
       ...,
       [0.73310108, 0.26689892],
       [0.22760011, 0.77239989],
       [0.830135  , 0.169865  ]])

In [45]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Punjab Kings',
 'Chennai Super Kings',
 'Rajasthan Royals']

In [46]:
final_df['city'].unique()

array(['Kolkata', 'Jaipur', 'Mumbai', 'Chennai', 'Dharamsala', 'Mohali',
       'Chandigarh', 'Bengaluru', 'Pune', 'Hyderabad', 'Delhi',
       'Bangalore', 'Ahmedabad', 'Durban', 'Nagpur', 'Ranchi', 'Raipur',
       'Abu Dhabi', 'Johannesburg', 'Centurion', 'Cuttack',
       'Visakhapatnam', 'Port Elizabeth', 'Indore', 'Kimberley', nan,
       'Bloemfontein', 'Sharjah', 'Cape Town', 'East London'],
      dtype=object)

In [47]:
# store the pipe made and then use it when using it with website 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))